## Imports


In order for us to read the files, we need to import certain modules. Pandas and altair allow us to plot the data in the picologger file

In [ ]:
import os
import sys
import pandas as pd
import altair as alt
from datetime import datetime,date

In [ ]:
### package versions
packList=["pandas","altair","numpy","itkdb"]
for p in packList:
    if p in sys.modules.keys():
        mod=sys.modules.get(p)
        print(mod.__name__+" : "+mod.__version__)

## Read in Data

In [ ]:
### list files in above directory
thisDir = os.getcwd()
os.listdir(thisDir+"/../")

## Format raw data
The following data takes in all of the files from the picologger folder,performs necessary calculations on them and puts them all in one csv file.

In [ ]:
### convert timestamp in filename
def TimeStampConverter(inStr, inPat):
    timeObj = None
    if inStr == "now":
        timeObj = datetime.now()
    else:
        timeObj = datetime.strptime(inStr, inPat)
    return timeObj.strftime("%Y-%m-%dT%H:%MZ")


In [ ]:
### derive values from picoLogger raw data
def perform_calculations(input_folder, output_file, timestamp_str):
    all_data = []

    # Convert the timestamp to the desired format
    timestamp = TimeStampConverter(timestamp_str, "%Y-%m-%d %H:%M:%S")

    for filename in os.listdir(input_folder):
        if filename.endswith('.csv'):
            file_path = os.path.join(input_folder, filename)
            data = pd.read_csv(file_path)

            # ... (the rest of your calculations remain unchanged) ...

            # Prepare the new DataFrame with desired columns and values
            result_data = pd.DataFrame({
                # ... (other columns remain unchanged) ...
                'timestamp': [timestamp]  # Wrap the timestamp value in a list
            })

            all_data.append(result_data)

    if all_data:
        final_result = pd.concat(all_data, ignore_index=True)
        final_result.to_csv(output_file, index=False)
        print(f"Calculation completed. Output saved to {output_file}.")


In [ ]:
### run conversion
input_folder = thisDir+"/../picolog_folder/"
output_file = thisDir+"/../output_file.csv"
timestamp_str = "2023-08-08 14:30:00"  # Replace this with your desired timestamp

perform_calculations(input_folder, output_file, timestamp_str)
